<a href="https://colab.research.google.com/github/JoseJ09/RNN_generate_text/blob/main/RNN_generate_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genracion de texto mediante el uso de RNN

## Alicia a traves del espejo - II
###  Lewis Carroll

In [ ]:
# Librerias

import sys
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Cargar el texto y convertir todo a minusculas
filename = "Alicia_esp.txt"

raw_text = open(filename, 'r', encoding='utf-8').read()

# Cambiar todo a minusculas
raw_text = raw_text.lower()

In [ ]:
# Ver el texto?
raw_text

'lewis carrol\n\nalicia a través del espejo\n\nla casa del espejo\n\ndesde luego hay una cosa de la que estamos bien seguros y es que el gatito blanco no tuvo absolutamente nada que ver con todo este enredo... fue enteramente culpa del gatito negro. en efecto, durante el último cuarto de hora, la vieja gata había sometido al minino blanco a una operación de aseo bien rigurosa (y hay que reconocer que la estuvo aguantando bastante bien); así que está bien claro que no pudo éste ocasionar el percance.\n\nla manera en que dina les lavaba la cara a sus mininos sucedía de la siguiente manera: primero sujetaba firmemente a la víctima con un pata y luego le pasaba la otra por toda la cara, sólo que a contrapelo, empezando por la nariz: y en este preciso momento, como antes decía, estaba dedicada a fondo al gatito blanco, que se dejaba hacer casi sin moverse y aún intentando ronronear... sin duda porque pensaba que todo aquello se lo estarían haciendo por su bien.\n\npero el gatito negro ya lo

In [ ]:
# Cambiar todos los caracteres a numeros
chars = sorted(list(set(raw_text)))

# ¿El orden de los caracteres es igual al alfabeto?
# ¿Que pasa con las puntuaciones y espacios?
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
# Observar la cantidad de letras y caracteres
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total del texto: ", n_chars)
print ("Total de vocabulario diferente: ", n_vocab)

Total del texto:  187278
Total de vocabulario diferente:  55


In [ ]:
# Definir la longitud de la cadena de caracteres a usar
seq_length = 100
dataX = []
dataY = []

# Construir las entradas y salidas de la red
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total de patrones de aprendizaje: ", n_patterns)

Total de patrones de aprendizaje:  187178


In [ ]:
# Escalar los datos entre 0 y 1 (Funcion sigmoide)

# Acomodar los datos de X de la forma [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar
X = X / float(n_vocab)
# One-hot encoder en las salidas
y = to_categorical(dataY)

In [ ]:
# Ver los datos
#X[0]
print(len(y[0]))
y[0]

55


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.], dtype=float32)

# Modelo con redes neuronales recurrentes

In [ ]:
# Definir el modelo de la RNN

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 256)          264192    
                                                                 
 dropout (Dropout)           (None, 100, 256)          0         
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 55)                14135     
                                                                 
Total params: 803,639
Trainable params: 803,639
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Como el modelo es pesado (gran tiempo de entrenamiento) se definen checkpoints

filepath="Perdida-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# Entrenar el modelo
model.fit(X,y, epochs=50, batch_size=1500, callbacks=callbacks_list)

Epoch 1/50
125/125 [==============================] - ETA: 0s - loss: 3.0876
Epoch 1: loss improved from inf to 3.08764, saving model to Perdida-01-3.0876.hdf5
125/125 [==============================] - 40s 247ms/step - loss: 3.0876
Epoch 2/50
125/125 [==============================] - ETA: 0s - loss: 2.9985
Epoch 2: loss improved from 3.08764 to 2.99853, saving model to Perdida-02-2.9985.hdf5
125/125 [==============================] - 33s 261ms/step - loss: 2.9985
Epoch 3/50
125/125 [==============================] - ETA: 0s - loss: 2.8583
Epoch 3: loss improved from 2.99853 to 2.85826, saving model to Perdida-03-2.8583.hdf5
125/125 [==============================] - 35s 277ms/step - loss: 2.8583
Epoch 4/50
125/125 [==============================] - ETA: 0s - loss: 2.7844
Epoch 4: loss improved from 2.85826 to 2.78437, saving model to Perdida-04-2.7844.hdf5
125/125 [==============================] - 35s 280ms/step - loss: 2.7844
Epoch 5/50
125/125 [==============================] - ET

# Generar nuevo texto basado en el set de entrenamiento

In [ ]:
# Cargar el modelo y compilarlo

filename = "Perdida-50-1.6351.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Para poder entenderlo necesitamos regresar de digitos a caracteres
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# Crear nuevo texto

# Selecciona de forma aleatorio un texto dentro del libro para dar como entrada
start = numpy.random.randint(0, len(dataX)-1)  # ¿Porque el -1?
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# Generacion de caracteres
for i in range(1000):
  x = numpy.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = numpy.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)

  # Se porpone un nuevo set de entrada
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
"  que contestes a mi pregunta -declaró la reina roja mirando su reloj-. abre bien la boca cuando habl "
a un cuento de la cabeza y con lus cesces de la cabeza y con la mano y la reina blanca sentado a la cabeza y con la mano y la reina blanca sentado a la cabeza y la conventera de la cabeza y contentó a alicia don el mundo de la cabeza y la reina blanca sentado a la cabeza y con la mano y la reina blanca sentado a la cabeza y la conventera de la cabeza y contentó a alicia don el mundo de la cabeza y la reina blanca sentado a la cabeza y con la mano y la reina blanca sentado a la cabeza y la conventera de la cabeza y contentó a alicia don el mundo de la cabeza y la reina blanca sentado a la cabeza y con la mano y la reina blanca sentado a la cabeza y la conventera de la cabeza y contentó a alicia don el mundo de la cabeza y la reina blanca sentado a la cabeza y con la mano y la reina blanca sentado a la cabeza y la conventera de la cabeza y contentó a alicia don el mundo de la ca